In [82]:
import pandas as pd
import numpy as np

In [83]:
rated_movies = pd.read_csv('filmes.csv')
movies = pd.read_csv('TMDB_all_movies.csv')
info_movies = pd.read_csv('TMDB_movie_dataset_v11.csv')

In [84]:
movies = pd.merge(movies, info_movies, on = 'id', how = 'inner')

In [85]:
movies.head()

,id,title_x,vote_average_x,vote_count_x,status_x,release_date_x,revenue_x,runtime_x,budget_x,imdb_id_x,...,original_title_y,overview_y,popularity_y,poster_path_y,tagline_y,genres_y,production_companies_y,production_countries_y,spoken_languages_y,keywords
0,2,Ariel,7.100,346.0,Released,1988-10-21,0.0,73.0,0.0,tt0094675,...,Ariel,After the coal mine he works at closes and his...,8.155,/ojDg0PGvs6R9xYFodRct2kdI6wC.jpg,NaN,"Drama, Comedy, Romance",Villealfa Filmproductions,Finland,Finnish,"prison, underdog, helsinki, finland, factory w..."
1,3,Shadows in Paradise,7.293,409.0,Released,1986-10-17,0.0,74.0,0.0,tt0092149,...,Varjoja paratiisissa,"Nikander, a rubbish collector and would-be ent...",5.946,/nj01hspawPof0mJmlgfjuLyJuRN.jpg,NaN,"Drama, Comedy, Romance",Villealfa Filmproductions,Finland,"English, Finnish, Swedish","helsinki, finland, salesclerk, garbage"
2,5,Four Rooms,5.862,2694.0,Released,1995-12-09,4257354.0,98.0,4000000.0,tt0113101,...,Four Rooms,It's Ted the Bellhop's first night on the job....,15.295,/75aHn1NOYXh4M7L5shoeQ6NGykP.jpg,Twelve outrageous guests. Four scandalous requ...,Comedy,"Miramax, A Band Apart",United States of America,English,"hotel, new year's eve, witch, bet, sperm, hote..."
3,6,Judgment Night,6.500,351.0,Released,1993-10-15,12136938.0,109.0,21000000.0,tt0107286,...,Judgment Night,"While racing to a boxing match, Frank, Mike, J...",13.564,/3rvvpS9YPM5HB2f4HYiNiJVtdam.jpg,Don't move. Don't whisper. Don't even breathe.,"Action, Crime, Thriller","Largo Entertainment, JVC",United States of America,English,"drug dealer, chicago, illinois, escape, one ni..."
4,8,Life in Loops (A Megacities RMX),7.500,27.0,Released,2006-01-01,0.0,80.0,42000.0,tt0825671,...,Life in Loops (A Megacities RMX),Timo Novotny labels his new project an experim...,1.587,/x7Sz339F2oC8mBf0DHCQpKizXaL.jpg,A Megacities remix.,Documentary,inLoops,Austria,"English, Hindi, Japanese, Russian, Spanish",megacities


In [86]:
movies = movies[['id','title_x','director','runtime_x','release_date_x' ,'adult','genres_y','keywords','overview_y','poster_path_y' ]]

In [87]:
movies.head()

,id,title_x,director,runtime_x,release_date_x,adult,genres_y,keywords,overview_y,poster_path_y
0,2,Ariel,Aki Kaurismäki,73.0,1988-10-21,False,"Drama, Comedy, Romance","prison, underdog, helsinki, finland, factory w...",After the coal mine he works at closes and his...,/ojDg0PGvs6R9xYFodRct2kdI6wC.jpg
1,3,Shadows in Paradise,Aki Kaurismäki,74.0,1986-10-17,False,"Drama, Comedy, Romance","helsinki, finland, salesclerk, garbage","Nikander, a rubbish collector and would-be ent...",/nj01hspawPof0mJmlgfjuLyJuRN.jpg
2,5,Four Rooms,"Quentin Tarantino, Robert Rodriguez, Allison A...",98.0,1995-12-09,False,Comedy,"hotel, new year's eve, witch, bet, sperm, hote...",It's Ted the Bellhop's first night on the job....,/75aHn1NOYXh4M7L5shoeQ6NGykP.jpg
3,6,Judgment Night,Stephen Hopkins,109.0,1993-10-15,False,"Action, Crime, Thriller","drug dealer, chicago, illinois, escape, one ni...","While racing to a boxing match, Frank, Mike, J...",/3rvvpS9YPM5HB2f4HYiNiJVtdam.jpg
4,8,Life in Loops (A Megacities RMX),Timo Novotny,80.0,2006-01-01,False,Documentary,megacities,Timo Novotny labels his new project an experim...,/x7Sz339F2oC8mBf0DHCQpKizXaL.jpg


In [88]:
movies.rename(columns = {'title_x': 'title', 'runtime_x': 'runtime', 'release_date_x': 'year' , 'overview_y': 'overview', 'genres_y': 'genres', 'poster_path_y' : 'poster_path' }, inplace = True)

In [89]:
movies['year'] = pd.to_datetime(movies['year'], errors='coerce').dt.year


In [90]:
def format_year(year):
    try:
        if pd.isnull(year):
            return ""
        return str(int(float(year)))
    except Exception:
        return str(year)
    
movies['year'] = movies['year'].apply(format_year)    

In [91]:
movies = movies.drop_duplicates(subset=['title', 'year']).reset_index(drop=True)

In [74]:
movies['title_year'] = movies['title'] + ' (' + movies['year'].astype(str) + ')'

In [75]:
movies.head()

,id,title,director,runtime,year,adult,genres,keywords,overview,poster_path,title_year
0,2,Ariel,Aki Kaurismäki,73.0,1988,False,"Drama, Comedy, Romance","prison, underdog, helsinki, finland, factory w...",After the coal mine he works at closes and his...,/ojDg0PGvs6R9xYFodRct2kdI6wC.jpg,Ariel (1988)
1,3,Shadows in Paradise,Aki Kaurismäki,74.0,1986,False,"Drama, Comedy, Romance","helsinki, finland, salesclerk, garbage","Nikander, a rubbish collector and would-be ent...",/nj01hspawPof0mJmlgfjuLyJuRN.jpg,Shadows in Paradise (1986)
2,5,Four Rooms,"Quentin Tarantino, Robert Rodriguez, Allison A...",98.0,1995,False,Comedy,"hotel, new year's eve, witch, bet, sperm, hote...",It's Ted the Bellhop's first night on the job....,/75aHn1NOYXh4M7L5shoeQ6NGykP.jpg,Four Rooms (1995)
3,6,Judgment Night,Stephen Hopkins,109.0,1993,False,"Action, Crime, Thriller","drug dealer, chicago, illinois, escape, one ni...","While racing to a boxing match, Frank, Mike, J...",/3rvvpS9YPM5HB2f4HYiNiJVtdam.jpg,Judgment Night (1993)
4,8,Life in Loops (A Megacities RMX),Timo Novotny,80.0,2006,False,Documentary,megacities,Timo Novotny labels his new project an experim...,/x7Sz339F2oC8mBf0DHCQpKizXaL.jpg,Life in Loops (A Megacities RMX) (2006)


In [93]:
rated_movies = rated_movies[['user_id', 'film_title', 'rating', 'film_year']]

In [94]:
rated_movies['film_year'] = rated_movies['film_year'].apply(format_year)    

In [ ]:
rated_movies['title_year'] = rated_movies['film_title'] + ' (' + movies['film_year'].astype(str) + ')'

In [92]:
rated_movies.head()

,user_id,film_title,film_url,rating,film_year
0,user_001,Scream 7,https://letterboxd.com/film/scream-7/,1.0,2026.0
1,user_001,Materialists,https://letterboxd.com/film/materialists/,NaN,2025.0
2,user_001,How to Train Your Dragon,https://letterboxd.com/film/how-to-train-your-...,6.0,2025.0
3,user_001,Predator: Killer of Killers,https://letterboxd.com/film/predator-killer-of...,8.0,2025.0
4,user_001,Echo Valley,https://letterboxd.com/film/echo-valley/,6.0,2025.0


In [14]:
movies = movies.fillna('')

movies['features'] = (
    movies['title'].astype(str) + ' ' +
    movies['director'].astype(str) + ' ' +
    movies['genres'].astype(str) + ' ' +
    movies['keywords'].astype(str) + ' ' +
    movies['overview'].astype(str) 
)

In [78]:
import re

def clean_text(text):
    if pd.isnull(text):
        return ""
    # Remove pontuação e coloca em minúsculas
    text = text.lower()
    text = re.sub(r'[.,]', '', text)
    return text

movies['features'] = movies['features'].apply(clean_text)

KeyError: 'features'

In [79]:
movie_name = "Moana"
resultado = movies[movies['title'] == movie_name]

In [80]:
resultado

,id,title,director,runtime,year,adult,genres,keywords,overview,poster_path,title_year
30398,45890,Moana,Alfredo Peyretti,181.0,2009,False,"Drama, TV Movie",NaN,The life story of Italian iconic pornographic ...,/nEDFaNVqmLl2UeUYvvkKDSge5Ed.jpg,Moana (2009)
65585,94727,Moana,"Frances H. Flaherty, Robert Flaherty",98.0,1926,False,Documentary,"south seas, indigenous, coconut, samoa, polyne...",Robert J. Flaherty's South Seas follow-up to N...,/dkSM2bfwYaqY0k6cEFwej1LRWAk.jpg,Moana (1926)
156480,268516,Moana,"Serge Arnoux, Pierre Pasquier, Bernard Gorki, ...",82.0,1959,False,Documentary,"south seas, hawaii, underwater",Documentary focused on underwater shootings an...,NaN,Moana (1959)
164551,277834,Moana,"Ron Clements, John Musker",107.0,2016,False,"Adventure, Comedy, Family, Animation","sailboat, sea, island, saving the world, ocean...","In Ancient Polynesia, when a terrible curse in...",/vNJFiwtsl7Twww7C0uM4qPnygNw.jpg,Moana (2016)
774109,1108427,Moana,Thomas Kail,0.0,2026,False,"Adventure, Comedy, Family, Fantasy","friendship, leaving one's family, musical, rem...","In Ancient Polynesia, when a terrible curse in...",/ys0jZr0quHERDUEoCboGQEKPvgQ.jpg,Moana (2026)
1001392,1397185,Moana,"Neon Wings, Jennifer Jackson",86.0,2025,False,"Animation, Fantasy, Family, Adventure",NaN,Plot is Being Covered,/14I1LD9kMCf7rEBitTdYQsCMZf4.jpg,Moana (2025)


In [49]:
movies.head()

,id,title,director,runtime,year,adult,genres,keywords,overview,poster_path
0,2,Ariel,Aki Kaurismäki,73.0,1988,False,"Drama, Comedy, Romance","prison, underdog, helsinki, finland, factory w...",After the coal mine he works at closes and his...,/ojDg0PGvs6R9xYFodRct2kdI6wC.jpg


In [17]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.neighbors import NearestNeighbors

vectorizer = CountVectorizer(token_pattern=r"(?u)\b\w+\b")
X = vectorizer.fit_transform(movies['features'])

knn = NearestNeighbors(metric='cosine', algorithm='brute')
knn.fit(X)

NearestNeighbors(algorithm='brute', metric='cosine')

In [18]:
def get_recommendations_by_content(title, n_recommendations=50):
    if title not in movies['title'].values:
        return f"Filme '{title}' não encontrado."
    
    idx = movies[movies['title'] == title].index[0]
    distances, indices = knn.kneighbors(X[idx], n_neighbors=n_recommendations+1)
    
    recommended_titles = [movies.iloc[i]['title'] for i in indices.flatten() if movies.iloc[i]['title'] != title]
    return recommended_titles[:n_recommendations]

In [19]:
def get_knn_ratings_recommendation(title, n_recommendations=5):

    content_recs = get_recommendations_by_content(title, n_recommendations=15)
    if not content_recs:
        return f"Filme '{title}' não encontrado."
    
    pivot = rated_movies.pivot_table(index='film_title', columns='user_id', values='rating').fillna(0)
    

    filtered_titles = [title] + [rec for rec in content_recs if rec in pivot.index]
    filtered_pivot = pivot.loc[filtered_titles]

    knn_ratings = NearestNeighbors(metric='cosine', algorithm='brute')
    knn_ratings.fit(filtered_pivot.values)
    
    idx = filtered_pivot.index.get_loc(title)
    distances, indices = knn_ratings.kneighbors([filtered_pivot.iloc[idx].values], n_neighbors=min(n_recommendations+1, len(filtered_titles)))

    recommended_titles = [filtered_pivot.index[i] for i in indices.flatten() if filtered_pivot.index[i] != title]
    return recommended_titles[:n_recommendations]

In [20]:
print(get_recommendations_by_content('Indiana Jones and the Last Crusade'))

['God Disposes', 'Eternal', 'The Old Cowboy', 'My Father Iqbal', 'Father and Son', 'Hitler: Beast of Berlin', 'Morning Star', 'Ghouls', 'The Cord of Life', 'The Han River', 'Hope in the Holy Land: Delving Beneath the Surface of the Israeli-Palestinian Conflict', 'Blood on the Asphalt', 'The Day of the Crows', 'Heart Beats of Long Ago', 'Consolation', 'Wind Back', 'The Grassland Whisper', 'The Devil in Sofia', 'No Mill No Meal', 'Won in the Fifth', "Majub's Journey", 'Der rote Reiter', 'The Misadventure of a French Gentleman Without Pants at the Zandvoort Beach', 'Glory of Legend', 'Premutos: The Fallen Angel', '家族ケチャップ', 'Fokak Meny', 'An Apple from Paradise', 'The Line Will Break', 'Tempest', 'The Stepmother', 'The Fisher-Maid', 'Summer in the Golden Valley', 'Traveler', 'The New Man', 'Youth on the palm of the imp', 'The Intruder', 'The Eremites', 'Rebels Of The Cities', 'Blood Loss', 'Alarm', 'The Old Man and the Bird', 'The Color of the Sun', 'Lost on the Branch', 'Jim is Fond of G

In [21]:
print(get_knn_ratings_recommendation('Indiana Jones and the Last Crusade', n_recommendations=5))

['Ghouls', 'Father and Son', 'Eternal']


In [81]:
import pickle

# Salva o DataFrame movies em um arquivo pickle
with open('movies_info.pkl', 'wb') as f:
    pickle.dump(movies, f)
# Salva o DataFrame rated_movies em um arquivo pickle
with open('rated_movies.pkl', 'wb') as f:
    pickle.dump(rated_movies, f)    

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

cosine_sim = cosine_similarity(X, X)